<a href="https://colab.research.google.com/github/kalebmes/CS376-Machine-Learning/blob/main/Phase_2_model_building_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use this notebook for the given task of the term project.

### ***Make sure to delete all text cells before submission to avoid an unncessary increase of Turnitin similarity. That is, leave only the code cells.***

We will use only **'train.csv'**. Load the 'train.csv' file into a Pands dataframe.

In [234]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [235]:
import numpy as np
import pandas as pd
import os
data_path = '/content/drive/MyDrive/CS492 term project datasets'
data_df = pd.read_csv(os.path.join(data_path, 'train.csv'))

Drop the 'ID', 'ProcessType', 'TransactionNature', 'Type', 'DeclarerID', 'ImporterID', 'SellerID', 'ExpressID', 'OriginCountry', 'DisplayIndicator', 'DutyRegime' attributes from the dataframe for simplicity. It doesn't mean that these attributes are unnucessary.

In [236]:
dropped_df = data_df.drop(columns=['ID', 'ProcessType', 'TransactionNature', 'Type', 'DeclarerID', 'ImporterID', 'SellerID', 'ExpressID', 'OriginCountry', 'DisplayIndicator','DutyRegime'])

Convert a date value in the 'IssueDateTime' attribute to its month value. For example, `2020-01-01` $\rightarrow$ `1`.

In [237]:
dropped_df['IssueDateTime'] = pd.to_datetime(dropped_df['IssueDateTime']).dt.month

Keep only the first two digits in the 'ClassificationID' attribute. For example, `9619001090` $\rightarrow$ `96`.

In [238]:
dropped_df['ClassificationID'] = (dropped_df['ClassificationID']//(10e7)).astype('int')

In [239]:
cat_cols = ['DeclarationOfficeID', 'PaymentType', 'BorderTransportMeans', 'ExportationCountry']
for col in cat_cols:
  dropped_df[col] = pd.factorize(dropped_df[col], sort=True)[0]

Please note that the 'DeclarationOfficeID', 'PaymentType', 'BorderTransportMeans', and 'ExportationCountry' attributes are categorical, even though some of them have numerical forms. Then, let's factorize these attributes. Here, for each unique value in an attribute, assign consecutive integer values in the increasing order of the original values. Let's call these newly assigned integers as *codes*. As an example with 'ExportationCountry', `AE` $\rightarrow$ `0`.

**Hint**: use `pd.factorize(train_df[...], sort=True)` for each of the attributes.

####Q1. What is the maximum code of the 'ExportationCountry' attribute?

####Q2. What is the maximum code of the 'DeclarationOfficeID' attribute?

In [240]:
#answer for question number 1
print(dropped_df['ExportationCountry'].max())
#answer for question number 2
print(dropped_df['DeclarationOfficeID'].max())

115
43


Convert the 'ClassificationID' attribute (categorical) to a set of asymmetric **binary** variables. Use `pd.get_dummies(...)`, and we do **not** want to introduce redundancy in this process (i.e., be caureful with the `drop_first` option). Keep the original dataframe and store this new one as another dataframe. 

#### Q3. What is the total number of columns of this **new** dataframe?


In [241]:
#answer for question number 3
new_df = dropped_df.copy()
new_df = pd.get_dummies(data=new_df, prefix='c_id_is', columns=['ClassificationID'], drop_first=True)
print('Number of columns:', new_df.shape[1])

Number of columns: 103


Split the dataframe (immediately after **Q2**, i.e., before executing `get_dummies`) into `train_part` that contains the rows from January through September and `test_part` that contains the rows from October through December.

Then, let's use `train_part` as the training set and `test_part` as the test set.

####Q4. How many examples (rows) are contained in `train_part`?

In [242]:
#answer for question number 4
train_part = dropped_df[dropped_df['IssueDateTime'] < 10]
test_part = dropped_df[dropped_df['IssueDateTime'] > 9]
print('number of training examples:', train_part.shape[0])

number of training examples: 29158


Let's consider a dumb classifier that **always** returns **0**. 

#### Q5. What is the accuracy of the dumb classifier on the test set `test_part`?

In [243]:
#answer for question number 5
dumb_classifier = test_part['Fake'] == 0
acc = len(test_part[dumb_classifier]) / len(test_part)
print(round(acc, 3))

0.784


Let's implement a **k-nearest neighbor** classifier. To ease your implementation, please use `KNeighborsClassifier` of the scikit-learn library. Refer to the scikit-learn [manual](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html). Use the default values for all parameters except `n_neighborsint` and `weights`.

Note that we want to weight points by the inverse of their distance.

#### Q6. Increase the number ($k$) of neighbors from **1** through **9**. At which value of $k$, is the accuracy of the classifier on the test set `test_part` maximized?

#### Q7. What is the accuracy achieved in **Q6**?

In [244]:
#answer for question number 6
from sklearn.neighbors import KNeighborsClassifier
features = list(train_part.columns)[:-1]
label = list(train_part.columns)[-1]
k_max_acc = 1
max_acc = 0
for k in range(1, 10):
  knn = KNeighborsClassifier(n_neighbors=k, weights='distance')
  knn.fit(train_part[features], train_part[label])
  acc = knn.score(test_part[features], test_part[label])
  print('For k =', k, ': acc = ', acc)
  if acc > max_acc:
    max_acc = acc
    k_max_acc = k
print('maximum accuracy is obtained at k =', k_max_acc)
#answer for question number 7
print('and the maximum accuracy is', round(max_acc, 3))

For k = 1 : acc =  0.8461449676823638
For k = 2 : acc =  0.8464912280701754
For k = 3 : acc =  0.8526084949215144
For k = 4 : acc =  0.8544552169898431
For k = 5 : acc =  0.8509926131117267
For k = 6 : acc =  0.8534164358264081
For k = 7 : acc =  0.8500692520775623
For k = 8 : acc =  0.8528393351800554
For k = 9 : acc =  0.8533010156971376
maximum accuracy is obtained at k = 4
and the maximum accuracy is 0.854


Repeat the training and test procedures using the other dataframe obtained in **Q3**.

#### Q8. Comparing the accuracy at the same value of $k$ as **Q6**~**Q7**, is the accuracy improved by using the other dataframe in **Q3**?

In [245]:
train_part = new_df[new_df['IssueDateTime'] < 10]
test_part = new_df[new_df['IssueDateTime'] > 9]
df = train_part.copy()
label = 'Fake'
df = df.drop(columns='Fake')
features = list(df.columns)
knn = KNeighborsClassifier(n_neighbors=k_max_acc, weights='distance')
knn.fit(train_part[features], train_part[label])
new_acc = knn.score(test_part[features], test_part[label])
print('Accuracy on the new dataframe:', new_acc)
print('Accuracy on the old dataframe:', max_acc)
print('Is the accuracy improved?', new_acc > max_acc)

Accuracy on the new dataframe: 0.8434903047091413
Accuracy on the old dataframe: 0.8544552169898431
Is the accuracy improved? False
